In [1]:
import os

import pandas as pd

from scripts.fasta_processing import dict_align_to_fasta
from scripts.datasets import check_transcript_count
from scripts.data_processing import analyze_exons, create_cassette, concat_cassette, dict_align_create, dict_align_info_analyze

## Введение

In [2]:
# читаем таблицы: с gene_id и taxonomy

column_names = ["tax_id", "org_name", "gene_id", "current_id", "status", "symbol", "aliases", "description",
                "other_designations", "map_location", "chromosome", "genomic_nucleotide_accession.version",
                "start_position_on_the_genomic_accession", "end_position_on_the_genomic_accession", "orientation",
                "exon_count", "to_delete_1", "to_delete_2"]

df = pd.read_csv("../all_nxf1_2.txt", sep="\t", skiprows=1, names=column_names, index_col=0)
df.drop(["to_delete_1", "to_delete_2"], axis=1, inplace=True)

df_taxonomy = pd.read_csv("../taxonomy/all_phylas_taxonomy.tsv", sep="\t", names=["taxid", "taxonomy"], index_col=0,
                          dtype={"taxid": int, "taxonomy": str})

# df_Protostomia = df_taxonomy[df_taxonomy.taxonomy.str.contains("Protostomia")]  # 83 вида, из которых 70 артроподы
# df_Protostomia.to_csv(f"../Protostomia_taxonomy.tsv", sep="\t", header=False)
# create_taxonomy(f"../Protostomia_taxonomy.tsv")

## Переопределение функций

In [3]:
def create_taxids(df, sub_phylum_list):
    taxids = {}
    for sub_phylum in sub_phylum_list:
        df_subset = df[df.taxonomy.str.contains(sub_phylum)]
        taxids[sub_phylum] = df_subset.index.tolist()
    return taxids

def create_many_cassettes(phylum: str, data: dict) -> dict:
    introns = {}
    for org_name, (df, exons_i) in data.items():
        cassette = create_cassette(phylum, org_name, df, exons_i=exons_i)
        introns[org_name] = concat_cassette(cassette, "i")
    return introns

def find_cassettes(phylum):
    prefix = "../datasets"
    postfix = "ncbi_dataset/data"
    nof = "exons.fa"
    ref_exon_len = [37]
    found_org_name_i = {}
    data = {}

    for org_name_i in os.listdir(f"{prefix}/{phylum}"):
        df_exons = analyze_exons(f"{prefix}/{phylum}/{org_name_i}/{postfix}/{nof}")
        condition = set(ref_exon_len) & set(df_exons.length.tolist())
        if condition:
            found_org_name_i[org_name_i] = condition

    for org_name_i in found_org_name_i:  # CHANGE
        df_exons = analyze_exons(f"{prefix}/{phylum}/{org_name_i}/{postfix}/{nof}")
        exon_37_idx = df_exons[df_exons["length"] == 37].index[0]
        exon_110_idx = exon_37_idx - 1
        data[f"{org_name_i}"] = (df_exons, [exon_110_idx, exon_37_idx])

    return data, found_org_name_i

def create_alignment(phylum, found_org_name_i, align_type = "cds_cassette"):
    org_names = list(found_org_name_i.keys())
    alignment_dict = dict_align_create(phylum, org_names, align_type)  # CHANGE
    alignment_dict_upd = {
        "_".join(key.split("_")[:-1]).capitalize(): value
        for key, value in alignment_dict.items()
    }  # обновляем названия организмов для красоты в Ugene
    os.makedirs(f"../Alignment/{phylum}", exist_ok=True)
    dict_align_to_fasta(alignment_dict_upd, f"../Alignment/{phylum}/{phylum}_cds_cassette.fa")
    dict_align_to_fasta(alignment_dict_upd, f"../Alignment/{phylum}/{phylum}_cds_cassette.aln")

def create_dict_align_info(sub_phylum_list):
    dict_align_info = {}
    for sub_phylum in sub_phylum_list:
        _, found_org_name_i = find_cassettes(sub_phylum)
        dict_align_info[sub_phylum] = found_org_name_i.keys()
    return dict_align_info

## Ecdysozoa

### Загрузка данных и создание кассет

In [4]:
# создаем сабсет таблицы для исследуемой филы
phylum = "Ecdysozoa"
df_for_analysis = df_taxonomy[df_taxonomy.taxonomy.str.contains(phylum)] # в этой таблице вручную определяем подпапки из принта следующих 2х команд

In [5]:
# создаем принт таксономии для исследуемой филы
# df_for_analysis.to_csv(f"../{phylum}_taxonomy.tsv", sep="\t", header=False)
# create_taxonomy(f"../{phylum}_taxonomy.tsv")

In [6]:
# создаем словарь taxids для последующей загрузки файлов
sub_phylum_list = ["Nematoda", "Scalidophora", "Chelicerata", "Crustacea", "Polyneoptera", "Paraneoptera", "Endopterygota"]
taxids = create_taxids(df_for_analysis, sub_phylum_list)

In [7]:
# загружаем файлы через datasets: gene, rna, cds, protein; exons (создаются автоматически после загрузки файлов)

# закомментировать, когда все загрузится!
# download_all_files_ncbi(df, taxids, phylas=list(taxids.keys()))

In [8]:
# определяем, у каких видов больше 1 транскрипта
# запускаем еще раз после прогона следующей ячейки, чтобы убедиться, что все удалилось
species_to_update = check_transcript_count(sub_phylum_list)

In [9]:
# удаляем все транскрипты, кроме 1-го - он, как правило, самый адекватный

# закомментировать, когда все обновится!
# update_data_for_species(species_to_update)

In [10]:
# создаем словарь data_phylum для видов, у которых в экзонах нашелся экзон размером 37 нт для дальнейшего создания кассет
for sub_phylum in sub_phylum_list:
    data, found_org_name_i = find_cassettes(sub_phylum)
    # и сразу создаем кассеты для найденных данных
    introns = create_many_cassettes(sub_phylum, data)
    # и еще создаем выравнивания cds_cassette для всех sub_phylum (на всякий случай)
    create_alignment(sub_phylum, found_org_name_i)

# удалил Chelicerata/centruroides_sculpturatus_3, вылезла ошибка

### Анализ интрона для всех выбранных сабфил

In [11]:
# сначала создаем словарь с данными
dict_align_info = create_dict_align_info(sub_phylum_list)

In [12]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(dict_align_info, "cds")
df_cds.equal_to_cds.unique()

array([ True])

In [13]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds

df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(dict_align_info, "cds_cassette")
df_cds_cassette

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
0,Nematoda,trichinella_spiralis_3,Trichinella_spiralis,84,False,83,1,417,83,datasets
1,Nematoda,brugia_malayi_2,Brugia_malayi,1095,False,956,139,243,110,datasets
2,Nematoda,caenorhabditis_elegans_0,Caenorhabditis_elegans,1089,False,896,193,106,110,datasets
3,Scalidophora,priapulus_caudatus_0,Priapulus_caudatus,1083,False,1082,1,2114,110,datasets
4,Chelicerata,varroa_destructor_5,Varroa_destructor,954,False,932,22,3077,110,datasets
...,...,...,...,...,...,...,...,...,...,...
65,Endopterygota,acromyrmex_echinatior_59,Acromyrmex_echinatior,1062,False,1061,1,438,113,datasets
66,Endopterygota,orussus_abietinus_39,Orussus_abietinus,1026,False,1025,1,74,113,datasets
67,Endopterygota,aphidius_gifuensis_42,Aphidius_gifuensis,1011,False,1010,1,240,113,datasets
68,Endopterygota,aphidius_gifuensis_43,Aphidius_gifuensis,1179,False,1016,163,246,113,datasets


In [14]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
11,Crustacea,amphibalanus_amphitrite_0,Amphibalanus_amphitrite,1173,False,1100,73,369,110,datasets
14,Crustacea,amphibalanus_amphitrite_1,Amphibalanus_amphitrite,1170,False,1100,70,392,110,datasets
17,Paraneoptera,melanaphis_sacchari_8,Melanaphis_sacchari,1185,False,1184,1,779,107,datasets
18,Paraneoptera,myzus_persicae_11,Myzus_persicae,1110,False,1043,67,70,107,datasets
19,Paraneoptera,acyrthosiphon_pisum_14,Acyrthosiphon_pisum,384,False,383,1,1196,107,datasets
20,Paraneoptera,diuraphis_noxia_13,Diuraphis_noxia,1080,False,1043,37,70,107,datasets
21,Paraneoptera,melanaphis_sacchari_5,Melanaphis_sacchari,1134,False,1037,97,71,107,datasets
22,Paraneoptera,sipha_flava_1,Sipha_flava,1038,False,1037,1,58,107,datasets
24,Paraneoptera,melanaphis_sacchari_9,Melanaphis_sacchari,1065,False,1028,37,77,107,datasets
25,Paraneoptera,diuraphis_noxia_12,Diuraphis_noxia,1239,False,1238,1,742,107,datasets


In [15]:
# удаляем дубликаты

# -----------------
# 19 - acyrthosiphon_pisum_14 - слишком длинный интрон в сравнении
# 14 - amphibalanus_amphitrite_1 - дубликат, старая сборка
# 68 - aphidius_gifuensis_43 - дубликат, старая сборка
# 20 - diuraphis_noxia_13 - дубликат
# 57 - leptopilina_heterotoma_32 - дубликат
# 26, 17, 24 - melanaphis_sacchari_7, melanaphis_sacchari_8, melanaphis_sacchari_9 - четвертипликат
# 18 - myzus_persicae_11 - дубликат
# 51, 53 - polistes_fuscatus_50, polistes_fuscatus_51 - трипликат
# 33, 28 - sipha_flava_0, sipha_flava_2 - трипликат
# 41 - vespula_pensylvanica_13 - дубликат
# -----------------

indices_to_drop = [19, 14, 68, 20, 57, 26, 17, 24, 18, 51, 53, 33, 28, 41]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [16]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/datasets_{phylum}_cds_cassette.tsv", sep="\t", index=False)

# 2 - Caenorhabditis_elegans - интрон полностью считывается!!!
# 31 - Rhopalosiphum_maidis - интрон полностью считывается!!!

## Spiralia

### Загрузка данных и создание кассет

In [17]:
# создаем сабсет таблицы для исследуемой филы
phylum = "Spiralia"
df_for_analysis = df_taxonomy[df_taxonomy.taxonomy.str.contains(phylum)] # в этой таблице вручную определяем подпапки из принта следующих 2х команд

In [18]:
# создаем принт таксономии для исследуемой филы
# df_for_analysis.to_csv(f"../{phylum}_taxonomy.tsv", sep="\t", header=False)
# create_taxonomy(f"../{phylum}_taxonomy.tsv")

In [19]:
# создаем словарь taxids для последующей загрузки файлов
sub_phylum_list = ["Platyhelminthes", "Bivalvia", "Gastropoda", "Brachiopoda"]
taxids = create_taxids(df_for_analysis, sub_phylum_list)

In [20]:
# загружаем файлы через datasets: gene, rna, cds, protein; exons (создаются автоматически после загрузки файлов)

# закомментировать, когда все загрузится!
# download_all_files_ncbi(df, taxids, phylas=list(taxids.keys()))

In [21]:
# определяем, у каких видов больше 1 транскрипта
# запускаем еще раз после прогона следующей ячейки, чтобы убедиться, что все удалилось
species_to_update = check_transcript_count(sub_phylum_list)

In [22]:
# удаляем все транскрипты, кроме 1-го - он, как правило, самый адекватный

# закомментировать, когда все обновится!
# update_data_for_species(species_to_update)

In [23]:
# создаем словарь data_phylum для видов, у которых в экзонах нашелся экзон размером 37 нт для дальнейшего создания кассет
for sub_phylum in sub_phylum_list:
    data, found_org_name_i = find_cassettes(sub_phylum)
    # и сразу создаем кассеты для найденных данных
    introns = create_many_cassettes(sub_phylum, data)
    # и еще создаем выравнивания cds_cassette для всех sub_phylum (на всякий случай)
    create_alignment(sub_phylum, found_org_name_i)

### Анализ интрона для всех выбранных сабфил

In [24]:
# сначала создаем словарь с данными
dict_align_info = create_dict_align_info(sub_phylum_list)

In [25]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(dict_align_info, "cds")
df_cds.equal_to_cds.unique()

array([ True])

In [26]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds

df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(dict_align_info, "cds_cassette")
df_cds_cassette

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
0,Platyhelminthes,schistosoma_haematobium_1,Schistosoma_haematobium,981,False,980,1,652,239,datasets
1,Bivalvia,magallana_gigas_2,Magallana_gigas,1035,False,1034,1,1537,110,datasets
2,Bivalvia,mya_arenaria_0,Mya_arenaria,1110,False,1109,1,1727,110,datasets
3,Bivalvia,crassostrea_virginica_1,Crassostrea_virginica,1110,False,1109,1,1613,110,datasets
4,Gastropoda,aplysia_californica_2,Aplysia_californica,1068,False,1067,1,4146,221,datasets
5,Gastropoda,gigantopelta_aegis_1,Gigantopelta_aegis,1050,False,1049,1,1869,110,datasets


In [27]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source


In [28]:
# удаляем дубликаты

# -----------------

# -----------------

# indices_to_drop = []
# df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [29]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/datasets_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Cnidaria (не Protostomia)

### Загрузка данных и создание кассет

In [30]:
# создаем сабсет таблицы для исследуемой филы
phylum = "Cnidaria"
df_for_analysis = df_taxonomy[df_taxonomy.taxonomy.str.contains(phylum)] # в этой таблице вручную определяем подпапки из принта следующих 2х команд

In [31]:
# создаем принт таксономии для исследуемой филы
# df_for_analysis.to_csv(f"../{phylum}_taxonomy.tsv", sep="\t", header=False)
# create_taxonomy(f"../{phylum}_taxonomy.tsv")

In [32]:
# создаем словарь taxids для последующей загрузки файлов
sub_phylum_list = ["Anthozoa", "Hydrozoa"]
taxids = create_taxids(df_for_analysis, sub_phylum_list)

In [33]:
# загружаем файлы через datasets: gene, rna, cds, protein; exons (создаются автоматически после загрузки файлов)

# закомментировать, когда все загрузится!
# download_all_files_ncbi(df, taxids, phylas=list(taxids.keys()))

In [34]:
# определяем, у каких видов больше 1 транскрипта
# запускаем еще раз после прогона следующей ячейки, чтобы убедиться, что все удалилось
species_to_update = check_transcript_count(sub_phylum_list)

In [35]:
# удаляем все транскрипты, кроме 1-го - он, как правило, самый адекватный

# закомментировать, когда все обновится!
# update_data_for_species(species_to_update)

In [36]:
# создаем словарь data_phylum для видов, у которых в экзонах нашелся экзон размером 37 нт для дальнейшего создания кассет
for sub_phylum in sub_phylum_list:
    data, found_org_name_i = find_cassettes(sub_phylum)
    # и сразу создаем кассеты для найденных данных
    introns = create_many_cassettes(sub_phylum, data)
    # и еще создаем выравнивания cds_cassette для всех sub_phylum (на всякий случай)
    create_alignment(sub_phylum, found_org_name_i)

### Анализ интрона для всех выбранных сабфил

In [37]:
# сначала создаем словарь с данными
dict_align_info = create_dict_align_info(sub_phylum_list)

In [38]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(dict_align_info, "cds")
df_cds.equal_to_cds.unique()

array([ True])

In [39]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds

df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(dict_align_info, "cds_cassette")
df_cds_cassette

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
0,Anthozoa,nematostella_vectensis_2,Nematostella_vectensis,1197,False,1172,25,991,116,datasets
1,Anthozoa,actinia_tenebrosa_1,Actinia_tenebrosa,1152,False,1142,10,173,116,datasets


In [40]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source


In [41]:
# удаляем дубликаты

# -----------------

# -----------------

# indices_to_drop = []
# df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [42]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/datasets_{phylum}_cds_cassette.tsv", sep="\t", index=False)